In [ ]:
import ultralytics
import os
from dotenv import load_dotenv
from roboflow import Roboflow
import cv2
from ultralytics import YOLO


In [ ]:
load_dotenv()
ultralytics.checks()

# Download dataset with 6369 butterflies.

In [ ]:
rf = Roboflow(api_key=os.getenv('ROBOFLOW_API_KEY'))
workspace = rf.workspace("firstwp-gzzwu")
project = workspace.project("butterfliesenhanced")
version = project.version(1)
dataset = version.download("yolov11")

# Train it using yolo-11-s and image size of 1024 with a decay for learning rate

In [ ]:
# !yolo task=detect mode=train model=yolo11s.pt data={dataset.location}/data.yaml epochs=100 imgsz=1024 lr0=0.001 weight_decay=0.0005 patience=5 plots=True

In [ ]:
!yolo task=detect mode=val model=runs/double_dataset_bigger/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
!yolo task=detect mode=predict model=runs/double_dataset_bigger/weights/best.pt conf=0.69 source=report_images_test save=True

In [ ]:
model_trained = YOLO("runs/double_dataset_bigger/weights/best.pt")

In [ ]:
actual_results = []
expected_predictions = []

For the challenge, to evaluate the metrics for the classification problem I used accuracy and f1 score.

In [ ]:
for expected_label, directory in [(0, os.getenv('NEGATIVE_IMAGES_CHALLENGE_PATH')), (1, os.getenv('POSITIVE_IMAGES_CHALLENGE_PATH'))]:
    list_images = os.listdir(directory)
    expected_predictions.extend([expected_label] * len(list_images))
    for image_name in list_images:
        results = model_trained(os.path.join(directory, image_name), conf=0.69)
        r = [1 for k in results if len(k.boxes.cls.cpu().numpy()) > 0]
        actual_results.append(1 if len(r) > 0 else 0)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
accuracy = accuracy_score(expected_predictions, actual_results)
f1 = f1_score(expected_predictions, actual_results, zero_division=0)
print(accuracy)
print(f1)

In [ ]:
sum(actual_results)

In [ ]:
sum(expected_predictions)

In [ ]:
len(expected_predictions)